# IBM watsonx.ai + CrewAI Workshop

This notebook shows how to:

- Connect to **IBM watsonx.ai** from Python
- Wrap watsonx.ai models so they can be used as a **custom LLM inside CrewAI**
- Build a small **multi‑agent workflow** (a research & writing crew) powered by watsonx.ai


## 0. Prerequisites

To run this notebook you need:

- Python 3.10+
- An IBM Cloud account with access to **watsonx.ai**
- A watsonx.ai **service instance**, **project**, and **API key**
- Basic familiarity with Python and virtual environments

> 💡 Run the cells in order from top to bottom the first time.


In [ ]:
# 1) Install required packages
# Run this once in each new environment.
# Remove the leading '%' if your environment does not support IPython magics.

%pip install -U "crewai[tools]" langchain-ibm ibm-watsonx-ai python-dotenv


In [ ]:
# 2) Quick version check (optional)

import sys, platform

print("Python:", sys.version.split()[0])
print("Platform:", platform.platform())

try:
    import crewai
    print("crewai:", crewai.__version__)
except Exception as e:
    print("crewai not imported yet:", e)

try:
    import langchain_ibm
    print("langchain-ibm:", getattr(langchain_ibm, "__version__", "unknown"))
except Exception as e:
    print("langchain-ibm not imported yet:", e)

try:
    import ibm_watsonx_ai
    print("ibm-watsonx-ai:", getattr(ibm_watsonx_ai, "__version__", "unknown"))
except Exception as e:
    print("ibm-watsonx-ai not imported yet:", e)


## 1. Configure IBM watsonx.ai credentials

You’ll need three pieces of information:

- **IBM Cloud API key** for watsonx.ai
- **Service URL** (depends on your region), for example:
  - `https://us-south.ml.cloud.ibm.com` (Dallas)
  - `https://eu-de.ml.cloud.ibm.com` (Frankfurt)
  - `https://eu-gb.ml.cloud.ibm.com` (London)
  - `https://jp-tok.ml.cloud.ibm.com` (Tokyo)
  - `https://au-syd.ml.cloud.ibm.com` (Sydney)
- **Project ID** for your watsonx.ai project

The next cell will ask for these values and store them in environment variables:

- `WATSONX_APIKEY` and `WATSONX_API_KEY` (both set for compatibility)
- `WATSONX_URL`
- `WATSONX_PROJECT_ID`


In [ ]:
# 3) Enter IBM watsonx.ai credentials

import os
from getpass import getpass

print("👉 Enter your IBM watsonx.ai credentials (they stay only in this notebook session).")

WATSONX_API_KEY = getpass("IBM Cloud API key for watsonx.ai: ")
WATSONX_URL = input("watsonx.ai URL (e.g. https://us-south.ml.cloud.ibm.com): ").strip()
WATSONX_PROJECT_ID = input("watsonx.ai Project ID: ").strip()

# Store in environment variables (some libraries expect one name, some the other)
os.environ["WATSONX_APIKEY"] = WATSONX_API_KEY
os.environ["WATSONX_API_KEY"] = WATSONX_API_KEY
os.environ["WATSONX_URL"] = WATSONX_URL
os.environ["WATSONX_PROJECT_ID"] = WATSONX_PROJECT_ID

print("\n✅ Environment variables set: WATSONX_APIKEY, WATSONX_API_KEY, WATSONX_URL, WATSONX_PROJECT_ID")


In [ ]:
WATSONX_URL = "https://us-south.ml.cloud.ibm.com"
os.environ["WATSONX_URL"] = WATSONX_URL

## 2. Quick sanity check: Chat with a watsonx.ai model

We’ll use the `ChatWatsonx` integration from **langchain-ibm** to make a simple
chat call and verify that credentials and networking work correctly.


In [ ]:
# 4) Simple chat call to watsonx.ai

from langchain_ibm import ChatWatsonx

# Pick any chat-capable model that is enabled in your region.
# You can adjust this later.
WATSONX_MODEL_ID = "ibm/granite-3-8b-instruct"

parameters = {
    "temperature": 0.3,
    "max_tokens": 256,
}

chat = ChatWatsonx(
    model_id=WATSONX_MODEL_ID,
    url=os.environ["WATSONX_URL"],
    project_id=os.environ["WATSONX_PROJECT_ID"],
    params=parameters,
)

messages = [
    ("system", "You are a concise technical assistant."),
    ("human", "In 3 bullet points, explain what CrewAI is."),
]

print("⏳ Calling watsonx.ai...")
response = chat.invoke(messages)
print("\n=== watsonx.ai response ===\n")
print(response.content)


## 3. Creating a CrewAI‑compatible watsonx LLM

CrewAI expects language models to follow a simple interface. For providers that
don’t have first‑class support in LiteLLM yet, CrewAI exposes a `BaseLLM` class
you can subclass.

We’ll now build a small wrapper around `ChatWatsonx` that implements this
interface so we can attach watsonx models directly to CrewAI agents.


In [ ]:
from typing import Any, Dict, List, Optional, Union

from crewai import BaseLLM
from langchain_ibm import ChatWatsonx


class WatsonxCrewAILLM(BaseLLM):
    """CrewAI-compatible wrapper around IBM watsonx.ai via ChatWatsonx.

    This implementation focuses on simple text-only chat.
    Tool calling could be added later by extending `call()`.
    """

    def __init__(
        self,
        model_id: str,
        url: str,
        project_id: str,
        api_key: Optional[str] = None,
        temperature: float = 0.3,
        max_tokens: int = 512,
        top_p: float = 1.0,
    ) -> None:
        # REQUIRED: call parent constructor with a model name & temperature
        super().__init__(model=model_id, temperature=temperature)

        import os

        if api_key is None:
            api_key = os.getenv("WATSONX_APIKEY") or os.getenv("WATSONX_API_KEY")

        if not api_key:
            raise ValueError(
                "No IBM watsonx API key provided. "
                "Set WATSONX_APIKEY/WATSONX_API_KEY or pass api_key explicitly."
            )

        # Make sure underlying SDK sees an API key
        os.environ.setdefault("WATSONX_APIKEY", api_key)
        os.environ.setdefault("WATSONX_API_KEY", api_key)

        self.url = url
        self.project_id = project_id
        self.model_id = model_id

        # Configure the underlying chat model
        self._chat = ChatWatsonx(
            model_id=model_id,
            url=url,
            project_id=project_id,
            params={
                "temperature": temperature,
                "max_tokens": max_tokens,
                "top_p": top_p,
            },
        )

    def call(
        self,
        messages: Union[str, List[Dict[str, str]]],
        tools: Optional[List[dict]] = None,
        callbacks: Optional[List[Any]] = None,
        available_functions: Optional[Dict[str, Any]] = None,
        **kwargs, # Add kwargs to accept unexpected arguments
    ) -> str:
        """Core entry point used by CrewAI.

        For this workshop we ignore tools/function-calling and just forward
        the content to watsonx.ai and return the model's text.
        """
        # Normalize input into something ChatWatsonx understands
        if isinstance(messages, str):
            # Just a plain user prompt
            chat_input = messages
        else:
            # Expect list of {"role": ..., "content": ...}
            processed: List[tuple] = []
            for m in messages:
                role = m.get("role", "user")
                content = m.get("content", "")
                if not content:
                    continue
                processed.append((role, content))

            # If we didn't manage to build a structured chat, fall back to flat text
            if processed:
                chat_input = processed
            else:
                chat_input = ""

        # Call watsonx.ai
        result = self._chat.invoke(chat_input)
        # langchain-ibm returns an AIMessage; we want the text content
        return getattr(result, "content", str(result))

    # Optional but recommended extra metadata methods

    def supports_function_calling(self) -> bool:
        """Tell CrewAI whether this LLM supports function/tool calling.

        We keep this `False` for the workshop to avoid wiring tool calling.
        """
        return False

    def get_context_window_size(self) -> int:
        """Approximate context window size in tokens.

        You can adjust this based on the specific model you choose.
        """
        return 8192


In [ ]:
# 6) Smoke test: use the custom LLM directly

watsonx_llm = WatsonxCrewAILLM(
    model_id=WATSONX_MODEL_ID,
    url=os.environ["WATSONX_URL"],
    project_id=os.environ["WATSONX_PROJECT_ID"],
    # api_key will be read from env if omitted
    temperature=0.3,
    max_tokens=256,
)

print("⏳ Calling watsonx via WatsonxCrewAILLM...")
reply = watsonx_llm.call("Say a one-sentence hello from watsonx.ai through CrewAI's BaseLLM.")
print("\n=== Custom LLM reply ===\n")
print(reply)


## 4. Building a small CrewAI workflow powered by watsonx.ai

We’ll build a simple three‑agent crew:

1. **Researcher** – gathers structured notes about a topic  
2. **Writer** – turns the notes into a tutorial‑style article  
3. **Editor** – polishes the article for a workshop audience  

All three agents will share the same `WatsonxCrewAILLM` instance so they all use
your watsonx.ai model.


In [ ]:
# 7) Choose a topic for the crew

topic = "Building multi-agent workflows with IBM watsonx.ai and CrewAI"

print("Current topic:", topic)
# Change the string above and re-run this cell to explore a different topic.


In [ ]:
# 8) Define CrewAI agents that use the watsonx-backed LLM

from crewai import Agent

researcher = Agent(
    role="AI Researcher",
    goal="Deeply research the given topic and produce clear, structured notes.",
    backstory=(
        "You are an expert AI research assistant. You excel at organizing complex "
        "information into concise bullet points that are easy to reuse."
    ),
    llm=watsonx_llm,
    verbose=True,
)

writer = Agent(
    role="Technical Writer",
    goal="Turn research notes into an engaging, practical tutorial article.",
    backstory=(
        "You are a patient technical writer who explains advanced AI concepts in "
        "a way that intermediate Python developers can understand."
    ),
    llm=watsonx_llm,
    verbose=True,
)

editor = Agent(
    role="Editor",
    goal=(
        "Improve clarity, structure, and tone of drafts so they are ready to be "
        "shared in a live workshop."
    ),
    backstory=(
        "You are a meticulous editor who focuses on correctness, structure, and "
        "beginner-friendly language."
    ),
    llm=watsonx_llm,
    verbose=True,
)


In [ ]:
# 9) Define tasks and how they depend on each other

from crewai import Task

research_task = Task(
    description=(
        f"Research the topic: '{topic}'.\n"
        "- Explain what IBM watsonx.ai is and its main capabilities.\n"
        "- Explain what CrewAI is and when you might use it.\n"
        "- List 8–12 concrete ideas for using watsonx.ai models inside multi-agent workflows.\n"
        "- Highlight any important security or cost considerations."
    ),
    expected_output=(
        "A markdown document with three sections: 'watsonx.ai overview', "
        "'CrewAI overview', and 'Use-case ideas', each with bullet points."
    ),
    agent=researcher,
)

writing_task = Task(
    description=(
        "Using the research notes, write a tutorial-style article about the topic.\n"
        "The article should cover:\n"
        "1. High-level overview.\n"
        "2. Architecture: where watsonx.ai fits vs CrewAI.\n"
        "3. Step-by-step guide to building a simple crew that calls a watsonx model.\n"
        "4. Practical tips and common pitfalls.\n"
        "Target length: 800–1200 words."
    ),
    expected_output=(
        "A markdown article with clear headings, short paragraphs, and at least "
        "one numbered list of steps."
    ),
    agent=writer,
    # Use the output of the research_task as context
    context=[research_task],
)

editing_task = Task(
    description=(
        "Take the draft article from the writer and polish it for a workshop\n"
        "audience. Focus on clarity, structure, and making it easy to follow in a\n"
        "live coding session.\n"
        "- Fix any obvious mistakes.\n"
        "- Make security best practices around API keys very explicit.\n"
        "- Ensure all code snippets are self-contained and well explained."
    ),
    expected_output=(
        "A polished markdown article ready to be shown in a training workshop."
    ),
    agent=editor,
    context=[writing_task],
)


In [ ]:
# 10) Create the crew and run the workflow

from crewai import Crew, Process

crew = Crew(
    agents=[researcher, writer, editor],
    tasks=[research_task, writing_task, editing_task],
    process=Process.sequential,  # run tasks one after another
    verbose=True,
)

print("⏳ Kicking off crew... this may take a little while.")
result = crew.kickoff()

# CrewAI typically returns an object with a `.raw` attribute, but we fall back
# gracefully in case the API changes.
final_text = getattr(result, "raw", str(result))

print("\n=== Final Workshop Article ===\n")
print(final_text)


## 5. Experiment: change the topic

To adapt this workflow to a different workshop or demo:

1. Go back to the **"Choose a topic"** cell and change the `topic` string.  
2. Re-run that cell.  
3. Re-run the **agent**, **task**, and **crew** cells (8–10).  

The same watsonx‑powered crew will now produce content for your new topic.


## 6. Next steps and extensions

Ideas for extending this notebook:

- **Add tools**: connect CrewAI tools (web search, file I/O, internal APIs) and
  wire them to watsonx by implementing function/tool calling in `WatsonxCrewAILLM`.
- **RAG workflows**: combine watsonx.ai with a vector store so agents can search
  over your own documentation or code.
- **Hierarchical crews**: introduce a manager agent that plans and delegates work
  instead of the simple sequential process used here.
- **Logging and observability**: connect CrewAI's tracing to your preferred
  monitoring stack to observe token usage and agent behaviour in production.

You can use this notebook as a starting point for your own internal workshops
or hands‑on labs around IBM watsonx.ai + CrewAI.
